# VISUALIZATION CONCENPT FOR SYNDROMIC DATA

1. Discharge Trends
    
    a. Figures summariseing patient  discharged (dead,referred, alive) by month or March April May
    
2. Patient Distibution by Age and Gender for the regions and month
3. Priority Indicators (Datasets recorded by health providers)

    a. COVID-19 vaccination status (period - March,April,Mya). \
        Recorded or not, Indeterminate,No, Undocumented,Yes
    b. HIV Status
        Negative,Positive,Undocumented,Unknown
    
    c. Comobordity at Admission
        Undocumented, No, Yes,
4. Duration of Hospital Stay
    In days , number of days, interquatile range, by period
5. Presenting symptoms at Admission
    Chest pain.cpugh,diarrhoea, difficulty in breathing, fever, headache, vomiting
6. Completion of Demographic measures
7. Recording of Vital Signs
8. Recording of Key Priority Measures
9. Laboratory tests done at admission (count and disease/condition testsd)
10. Disease patterns at discharge
11. Outcome at Discharge


    

In [83]:
%cd '/Users/jmwanga/Library/CloudStorage/OneDrive-KemriWellcomeTrust/Visualization/Kenya-COVID-19-Dashboard/syndromic_data/'

/Users/jmwanga/Library/CloudStorage/OneDrive-KemriWellcomeTrust/Visualization/Kenya-COVID-19-Dashboard/syndromic_data


In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.options.mode.chained_assignment = None
import numpy as np

data = pd.read_csv('./Syndromic_Surveillance_data.csv',low_memory=False)
data[['date_of_admission', 'date_of_discharge']] = data[['date_of_admission', 'date_of_discharge']].\
                                apply(pd.to_datetime, format="%Y-%m-%d")
data[(data['sex'] != 'Male') & (data['sex'] != 'Female')]
data = data[data['date_of_admission'] >'2022-12-31']
data.age_years.fillna(data.calculated_age,inplace=True)
data['month'] = data['date_of_admission'].apply(lambda x: x.strftime('%b-%y'))

#map regions
map_dict = {'Naivasha':'Central',
'Kiambu':'Central','Mbagathi':'Central','Mama_Lucy':'Central','JOOTRH':'Western','Kisumu':'Western','Kakamega':'Western','Busia':'Western',
'Kitale':'Western','Bungoma':'Western','Kisii':'Western'}

data['region'] = data['hospital'].map(map_dict)

#group age groups
def age_group(dataframe,column):
        '''Function to generate age groups. returns age groups column in the data frame'''
        dataframe['age_groups'] = pd.qcut(dataframe[column],
                                       [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,1],
                                       labels=['<10','11-20','21-30','31-40','41-50','51-60','61-70','>70'])
        return dataframe
    
data = age_group(data,'age_years')
data['duration'] = (data['date_of_discharge']-data['date_of_admission'])/np.timedelta64(1,'D')

total_central = len(data[data['region'] == 'Central'])
total_western = len(data[data['region'] == 'Western'])
sum_all = total_central + total_western

In [2]:
df_test = data[['outcome','status_at_discharge','date_of_admission','date_of_discharge','sex','age_years','month','region','d1_present','cause_of_death',\
                'disease_specific_tests___1','disease_specific_tests___2','disease_specific_tests___4','disease_specific_tests___5','microbiology___1',
                'biochemistry___3','haematology___1','microbiology___5']]


df_test.rename(columns={'disease_specific_tests___1':'Malaria_test','disease_specific_tests___2':'TB_test','disease_specific_tests___4':'HIV_Test',
                        'disease_specific_tests___5':'COVID-19_Test','microbiology___1':'Urinalysis','microbiology___5':'stool_analysis',
                        'biochemistry___3':'Blood_Glucose','haematology___1':'Full_blood_count'},
               
                inplace=True)

df_test.head()

,outcome,status_at_discharge,date_of_admission,date_of_discharge,sex,age_years,month,region,d1_present,cause_of_death,Malaria_test,TB_test,HIV_Test,COVID-19_Test,Urinalysis,Blood_Glucose,Full_blood_count,stool_analysis
0,Alive,Discharged,2023-05-08,2023-05-24,Male,23.0,May-23,Western,NaN,NaN,Checked,Unchecked,Unchecked,Unchecked,Checked,Unchecked,Checked,Unchecked
1,Alive,Discharged,2023-03-27,2023-04-01,Male,52.0,Mar-23,Western,NaN,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Checked,Unchecked
2,Alive,Discharged,2023-03-12,2023-04-04,Male,32.0,Mar-23,Western,NaN,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Checked,Unchecked
3,Alive,Discharged,2023-03-31,2023-04-03,Male,37.0,Mar-23,Western,NaN,NaN,Checked,Unchecked,Unchecked,Unchecked,Checked,Unchecked,Checked,Unchecked
4,Alive,Discharged,2023-03-31,2023-04-03,Male,63.0,Mar-23,Western,NaN,NaN,Checked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Checked,Checked


In [49]:
df = df_test[['month','TB_test','Malaria_test','HIV_Test','COVID-19_Test','Urinalysis','Blood_Glucose','Full_blood_count','stool_analysis']]#.to_dict('records')
df = df.groupby('month').apply(lambda x: x.eq('Checked').sum())
df = df.drop('month',axis=1)
df = df.apply(lambda x :round(100*x/5412),1)
#df = df.T.reset_index()#.columns

df#[['index']]#.columns#drop('region')

,TB_test,Malaria_test,HIV_Test,COVID-19_Test,Urinalysis,Blood_Glucose,Full_blood_count,stool_analysis
month,,,,,,,,
Apr-23,5.0,14.0,6.0,4.0,9.0,15.0,39.0,4.0
Feb-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jan-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jun-23,0.0,2.0,1.0,0.0,1.0,1.0,5.0,0.0
Mar-23,1.0,2.0,2.0,1.0,3.0,3.0,7.0,1.0
May-23,4.0,15.0,6.0,2.0,7.0,13.0,39.0,3.0


In [51]:
px.imshow(df.T)

In [47]:
px.bar(df, y = ['Apr-23','Feb-23','Jan-23','Jun-23','Mar-23','May-23'] , x= 'index',barmode='group')

In [91]:
def presenting_symptoms(value):
    df = df_test[df_test[value] == 'Checked']
    sum_len = len(df)
    df = df.groupby('region')[[value]].count()
    df[value] = df[value].apply(lambda x:(x/sum_len))
    return df

malaria_test = presenting_symptoms('Malaria_test')
#tb_test = presenting_symptoms('TB_test')
#hiv_test = presenting_symptoms('HIV_Test')
presenting_symptoms('HIV_Test')


,HIV_Test
region,
Central,0.500611
Western,0.457875


In [186]:
df_fever = presenting_symptoms('fever')
df_vomiting = presenting_symptoms('vomiting')
df_dysuria = presenting_symptoms('dysuria')
df_losscon = presenting_symptoms('loss_of_consciousness')

symp_data = pd.merge(df_headach,df_diarrhoea,on='region').\
    merge(df_cough,on='region').merge(df_chestpain,on='region').\
            merge(df_vomiting,on='region').merge(df_fever,on='region').\
                        merge(df_dysuria,on='region').merge(df_losscon,on='region')

In [168]:
status_at_discharge = df_data[['status_at_discharge']].value_counts().rename_axis('status').reset_index(name='counts')
status_at_discharge['prop'] = status_at_discharge['counts']/status_at_discharge['counts'].sum()*100
status_at_discharge

,status,counts,prop
0,Discharged,4025,95.764930
1,Referred,95,2.260290
2,Absconded,53,1.261004
3,DAMA,30,0.713776


In [150]:
df_data[df_data['outcome'] == 'Alive']['status_at_discharge'].value_counts()

Discharged    4025
Referred        95
Absconded       53
DAMA            30
Name: status_at_discharge, dtype: int64

In [172]:
categories = ['Category A', 'Category B', 'Category C']
start_dates = ['2023-01-01', '2023-01-02', '2023-01-03']
end_dates = ['2023-01-06', '2023-01-07', '2023-01-08']

# Create the dumbbell plot
fig = go.Figure()

# Add lines connecting start and end dates horizontally for each category
for cat, start, end in zip(categories, start_dates, end_dates):
    fig.add_trace(go.Scatter(
        x=[start, end],
        y=[cat, cat],
        mode='lines+markers',
        line=dict(color='blue', width=2),
        marker=dict(color='red', size=10),
        text=[start, end],
        hoverinfo='x+text+y',
        name=cat
    ))

# Set axis labels and title
fig.update_layout(
    title='Dumbbell Plot: Start Date to End Date',
    xaxis_title='Date',
    yaxis_title='Categories',
)

# Show the plot
fig.show()

In [170]:
import plotly.graph_objects as go

# Sample data (replace with your own data)
categories = ['Category A', 'Category B', 'Category C']
start_dates = ['2023-01-01', '2023-01-02', '2023-01-03']
end_dates = ['2023-01-06', '2023-01-07', '2023-01-08']

### Gender, Age, Region Distribution

# Data Clarifications

How important is the Medical Admission Record form used?

There exists:

    1. 3 entries with no gender information.

    
    2. 4 entries with dates before 2023-01-01

    3. How do we extract region information? No variable to define region. Do we use hospital or subcounty?

    5. Why weight specified  Min: 40, Max: 150

    6. On the outcome section, what is D1_present?





In [22]:
data1 = pd.read_table('../../Kenya-COVID-19-Dashboard/syndromic_data/kenya_data/1689336148515.metadata.tsv')
data2 = pd.read_table('../../Kenya-COVID-19-Dashboard/syndromic_data/kenya_data/1689336178136.metadata.tsv')
data3 = pd.read_table('../../Kenya-COVID-19-Dashboard/syndromic_data/kenya_data/1689336349172.metadata.tsv')

data1 = data1[['strain','originating_lab','submitting_lab']]
data2 = data2[['strain','originating_lab','submitting_lab']]
data3 = data3[['strain','originating_lab','submitting_lab']]

df = pd.concat([data1,data2,data3])

In [25]:
len(df)

12630

In [24]:
df.to_csv('../syndromic_data/kenya_data/labs.txt',sep='\t',index=False)